In [1]:
import re
import nltk
import collections
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
stop_words = set(stopwords.words('english'))

In [2]:
with open('train_5500.label.txt', 'rb') as f:
    data = f.readlines()

In [3]:
def clean_text(text):
    text = str(text)
    temp = text.split(":")
    label = temp[0][2:]
    text = (" ").join(temp[1].split()[1:])
    text = re.sub(r"@[A-Za-z0-9]+", ' ', text)
    text = re.sub(r'[^\w\s]','',text)
    text = re.sub(r"https?://[A-Za-z0-9./]+", ' ', text)
    text = re.sub(r"[^a-zA-z.!?'0-9]", ' ', text)
    text = re.sub('\t', ' ',  text)
    text = re.sub(r" +", ' ', text)
    return(text[:-2].lower(), label)

In [4]:
line = str(data[0])

temp = line.split(":")

In [5]:
temp[0][2:]

'DESC'

In [6]:
(" ").join(temp[1].split()[1:])

"How did serfdom develop in and then leave Russia ?\\n'"

In [7]:
X_train = []
Y_train = []
for line in data:
    sent, lab = clean_text(str(line))
    X_train.append(sent)
    Y_train.append(lab)

In [8]:
Y_train[0]

'DESC'

In [9]:
from nltk.util import ngrams

In [10]:
def generate_N_grams(words,ngram=1, freq=500): 
    res= ngrams(words,ngram)
    mostFreq = collections.Counter(res)
    ans=mostFreq.most_common(freq)
    return ans

In [11]:
words= []
for sent, label in zip(X_train, Y_train):
    words.extend([word for word in sent.split(" ") if word not in set(stopwords.words('english'))])
print(words)

['serfdom', 'develop', 'leave', 'russia', 'films', 'featured', 'character', 'popeye', 'doyle', 'find', 'list', 'celebrities', 'real', 'names', 'fowl', 'grabs', 'spotlight', 'chinese', 'year', 'monkey', 'full', 'form', 'com', 'contemptible', 'scoundrel', 'stole', 'cork', 'lunch', 'team', 'baseball', 'st', 'louis', 'browns', 'become', 'oldest', 'profession', 'liver', 'enzymes', 'name', 'scarfaced', 'bounty', 'hunter', 'old', 'west', 'ozzy', 'osbourne', 'born', 'heavier', 'objects', 'travel', 'downhill', 'faster', 'pride', 'yankees', 'killed', 'gandhi', 'considered', 'costliest', 'disaster', 'insurance', 'industry', 'ever', 'faced', 'sprawling', 'us', 'state', 'boasts', 'airports', 'repealed', 'amendment', 'us', 'constitution', 'deal', 'many', 'jews', 'executed', 'concentration', 'camps', 'wwii', 'nine', 'inch', 'nails', 'annotated', 'bibliography', 'date', 'boxing', 'day', 'articles', 'clothing', 'tokens', 'monopoly', 'name', '11', 'famous', 'martyrs', 'olympic', 'motto', 'origin', 'name

In [17]:
top_ngrams = [w[0][0] for w in generate_N_grams(words,1,500)]

In [18]:
top_ngrams

['many',
 'name',
 'first',
 'world',
 'us',
 'country',
 'city',
 'people',
 'get',
 'find',
 'called',
 'state',
 'year',
 'american',
 'president',
 'mean',
 'war',
 'two',
 'largest',
 'fear',
 'new',
 'made',
 'much',
 'long',
 'origin',
 'word',
 'company',
 'used',
 'tv',
 'movie',
 'kind',
 'film',
 'famous',
 'known',
 'one',
 'day',
 'take',
 'stand',
 'game',
 'make',
 'live',
 'invented',
 'best',
 'book',
 'time',
 'john',
 'color',
 'man',
 'play',
 'come',
 'states',
 'term',
 'show',
 'last',
 'named',
 'south',
 'countries',
 'wrote',
 'america',
 'home',
 'character',
 'team',
 'baseball',
 'old',
 'star',
 'call',
 'number',
 'difference',
 'english',
 'river',
 'use',
 'king',
 'born',
 'died',
 'highest',
 'work',
 'four',
 'song',
 'novel',
 'played',
 'information',
 'united',
 'three',
 'become',
 'capital',
 'common',
 'dog',
 'years',
 'actor',
 'average',
 'college',
 'names',
 'body',
 'earth',
 'actress',
 'say',
 'black',
 'population',
 'would',
 'water',

In [13]:
def gini_impurity(y):
    '''
    Given a Pandas Series, it calculates the Gini Impurity. 
    y: variable with which calculate Gini Impurity.
    '''
    if isinstance(y, pd.Series):
        p = y.value_counts()/y.shape[0]
        gini = 1-np.sum(p**2)
        return(gini)

    else:
        raise('Object must be a Pandas Series.')

gini_impurity(data.Gender) 

AttributeError: 'list' object has no attribute 'Gender'

In [ ]:
def entropy(y):
    '''
    Given a Pandas Series, it calculates the entropy. 
    y: variable with which calculate entropy.
    '''
    if isinstance(y, pd.Series):
        a = y.value_counts()/y.shape[0]
        entropy = np.sum(-a*np.log2(a+1e-9))
        return(entropy)

    else:
        raise('Object must be a Pandas Series.')

entropy(data.Gender)  

In [ ]:
import itertools

def categorical_options(a):
    '''
    Creates all possible combinations from a Pandas Series.
    a: Pandas Series from where to get all possible combinations. 
    '''
    a = a.unique()

    opciones = []
    for L in range(0, len(a)+1):
        for subset in itertools.combinations(a, L):
            subset = list(subset)
            opciones.append(subset)

    return opciones[1:-1]

def max_information_gain_split(x, y, func=entropy):
    '''
    Given a predictor & target variable, returns the best split, the error and the type of variable based on a selected cost function.
    x: predictor variable as Pandas Series.
    y: target variable as Pandas Series.
    func: function to be used to calculate the best split.
    '''

    split_value = []
    ig = [] 

    numeric_variable = True if x.dtypes != 'O' else False

    # Create options according to variable type
    if numeric_variable:
        options = x.sort_values().unique()[1:]
    else: 
        options = categorical_options(x)

    # Calculate ig for all values
    for val in options:
        mask =   x < val if numeric_variable else x.isin(val)
        val_ig = information_gain(y, mask, func)
        # Append results
        ig.append(val_ig)
        split_value.append(val)

    # Check if there are more than 1 results if not, return False
    if len(ig) == 0:
        return(None,None,None, False)

    else:
        # Get results with highest IG
        best_ig = max(ig)
        best_ig_index = ig.index(best_ig)
        best_split = split_value[best_ig_index]
        return(best_ig,best_split,numeric_variable, True)


    weight_ig, weight_slpit, _, _ = max_information_gain_split(data['Weight'], data['obese'],)  


    print(
    "The best split for Weight is when the variable is less than ",
    weight_slpit,"\nInformation Gain for that split is:", weight_ig
    )